LC borrower loan status prediction

step 1: Load and prepare the data set


In [33]:
## Import packages
library(ggplot2)
library(repr)
library(dplyr)
options(repr.plot.width=4, repr.plot.height=4)

In [34]:
credit = read.csv('LoanStats3a.csv', header=TRUE, stringsAsFactors=FALSE)
str(credit)
head(credit)

'data.frame':	42542 obs. of  145 variables:
 $ id                                        : chr  "" "" "" "" ...
 $ member_id                                 : logi  NA NA NA NA NA NA ...
 $ loan_amnt                                 : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt                               : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt_inv                           : num  4975 2500 2400 10000 3000 ...
 $ term                                      : chr  " 36 months" " 60 months" " 36 months" " 36 months" ...
 $ int_rate                                  : chr  "10.65%" "15.27%" "15.96%" "13.49%" ...
 $ installment                               : num  162.9 59.8 84.3 339.3 67.8 ...
 $ grade                                     : chr  "B" "C" "C" "C" ...
 $ sub_grade                                 : chr  "B2" "C4" "C5" "C1" ...
 $ emp_title                                 : chr  "" "Ryder" "" "AIR RESOURCES BOARD

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
,NA,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NA,,Cash,N,,,,NA,NA,NA
,NA,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NA,,Cash,N,,,,NA,NA,NA
,NA,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NA,,Cash,N,,,,NA,NA,NA
,NA,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NA,,Cash,N,,,,NA,NA,NA
,NA,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,NA,,Cash,N,,,,NA,NA,NA
,NA,5000,5000,5000,36 months,7.90%,156.46,A,A4,...,NA,,Cash,N,,,,NA,NA,NA


data analysis:

select interested features

In [35]:
selected.columns <- c('loan_status', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
                     'grade','emp_length', 'home_ownership', 'annual_inc', 'verification_status',
                     'issue_d', 'purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
                     'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
                     'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv')
credit <- credit[,selected.columns]

In [36]:
head(credit)
str(credit)

loan_status,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv
Fully Paid,5000,5000,4975,36 months,10.65%,162.87,B,10+ years,RENT,...,NA,NA,3,0,13648,83.70%,9,f,0,0
Charged Off,2500,2500,2500,60 months,15.27%,59.83,C,< 1 year,RENT,...,NA,NA,3,0,1687,9.40%,4,f,0,0
Fully Paid,2400,2400,2400,36 months,15.96%,84.33,C,10+ years,RENT,...,NA,NA,2,0,2956,98.50%,10,f,0,0
Fully Paid,10000,10000,10000,36 months,13.49%,339.31,C,10+ years,RENT,...,35,NA,10,0,5598,21%,37,f,0,0
Fully Paid,3000,3000,3000,60 months,12.69%,67.79,B,1 year,RENT,...,38,NA,15,0,27783,53.90%,38,f,0,0
Fully Paid,5000,5000,5000,36 months,7.90%,156.46,A,3 years,RENT,...,NA,NA,9,0,7963,28.30%,12,f,0,0


'data.frame':	42542 obs. of  28 variables:
 $ loan_status           : chr  "Fully Paid" "Charged Off" "Fully Paid" "Fully Paid" ...
 $ loan_amnt             : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt           : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt_inv       : num  4975 2500 2400 10000 3000 ...
 $ term                  : chr  " 36 months" " 60 months" " 36 months" " 36 months" ...
 $ int_rate              : chr  "10.65%" "15.27%" "15.96%" "13.49%" ...
 $ installment           : num  162.9 59.8 84.3 339.3 67.8 ...
 $ grade                 : chr  "B" "C" "C" "C" ...
 $ emp_length            : chr  "10+ years" "< 1 year" "10+ years" "10+ years" ...
 $ home_ownership        : chr  "RENT" "RENT" "RENT" "RENT" ...
 $ annual_inc            : num  24000 30000 12252 49200 80000 ...
 $ verification_status   : chr  "Verified" "Source Verified" "Not Verified" "Source Verified" ...
 $ issue_d               : chr  "Dec-11" "Dec-

remove duplicate instances and instances with wrong label

In [37]:
credit <- filter(credit, loan_status!='')
credit$loan_status <- as.factor(credit$loan_status)
summary(credit$loan_status)
credit = distinct(credit)
dim(credit)

Charged Off 
                                               5670 
Does not meet the credit policy. Status:Charged Off 
                                                761 
 Does not meet the credit policy. Status:Fully Paid 
                                               1988 
                                         Fully Paid 
                                              34116

[1] 42535    28

Feature Engineering & Feature Selection
1 Transform column data type.
2 Complete default value of some features.
2 Clean missing values.
3 Remove features with too many missing value.
4 

In [38]:
#credit$int_rate<-as.character(credit$int_rate)
credit$int_rate<-as.numeric(sub("%", "", credit$int_rate))/100
credit$int_rate[1:5]
#credit$revol_util<-as.character(credit$revol_util)
credit$revol_util<-as.numeric(sub("%", "", credit$revol_util))/100
credit$revol_util[1:5]

[1] 0.1065 0.1527 0.1596 0.1349 0.1269

[1] 0.837 0.094 0.985 0.210 0.539

In [39]:
credit$issue_d<-Sys.Date() - as.Date(as.character(credit$issue_d), format='%b-%y')
credit$earliest_cr_line <-Sys.Date() - as.Date(as.character(credit$earliest_cr_line), format='%b-%y')

In [40]:
names(credit)
str(credit)

[1] "loan_status"            "loan_amnt"              "funded_amnt"           
 [4] "funded_amnt_inv"        "term"                   "int_rate"              
 [7] "installment"            "grade"                  "emp_length"            
[10] "home_ownership"         "annual_inc"             "verification_status"   
[13] "issue_d"                "purpose"                "dti"                   
[16] "delinq_2yrs"            "earliest_cr_line"       "inq_last_6mths"        
[19] "mths_since_last_delinq" "mths_since_last_record" "open_acc"              
[22] "pub_rec"                "revol_bal"              "revol_util"            
[25] "total_acc"              "initial_list_status"    "out_prncp"             
[28] "out_prncp_inv"

'data.frame':	42535 obs. of  28 variables:
 $ loan_status           : Factor w/ 4 levels "Charged Off",..: 4 1 4 4 4 4 4 4 1 1 ...
 $ loan_amnt             : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt           : int  5000 2500 2400 10000 3000 5000 7000 3000 5600 5375 ...
 $ funded_amnt_inv       : num  4975 2500 2400 10000 3000 ...
 $ term                  : chr  " 36 months" " 60 months" " 36 months" " 36 months" ...
 $ int_rate              : num  0.106 0.153 0.16 0.135 0.127 ...
 $ installment           : num  162.9 59.8 84.3 339.3 67.8 ...
 $ grade                 : chr  "B" "C" "C" "C" ...
 $ emp_length            : chr  "10+ years" "< 1 year" "10+ years" "10+ years" ...
 $ home_ownership        : chr  "RENT" "RENT" "RENT" "RENT" ...
 $ annual_inc            : num  24000 30000 12252 49200 80000 ...
 $ verification_status   : chr  "Verified" "Source Verified" "Not Verified" "Source Verified" ...
 $ issue_d               : 'difftime' num  NA NA NA NA

In [51]:
#credit<-subset(credit, term!='' && total_acc!='')
credit$initial_list_status <- as.factor(credit$initial_list_status)
credit$term <- as.factor(credit$term)
credit$grade <- as.factor(credit$grade)
credit$emp_length  <- as.factor(credit$emp_length )
credit$home_ownership  <- as.factor(credit$home_ownership )
credit$verification_status  <- as.factor(credit$verification_status )
credit$purpose  <- as.factor(credit$purpose )
credit$delinq_2yrs <- as.numeric(credit$delinq_2yrs)
default_col <- c('mths_since_last_delinq','mths_since_last_record')
credit[,default_col] <- lapply(credit[,default_col], function(x){ifelse(x==NA, 0 ,x)})
#credit <- credit[complete.cases(credit$mths_since_last_delinq),]
credit$initial_list_status   <- as.factor(credit$initial_list_status  )
str(credit)

'data.frame':	0 obs. of  28 variables:
 $ loan_status           : Factor w/ 4 levels "Charged Off",..: 
 $ loan_amnt             : int 
 $ funded_amnt           : int 
 $ funded_amnt_inv       : num 
 $ term                  : Factor w/ 2 levels " 36 months"," 60 months": 
 $ int_rate              : num 
 $ installment           : num 
 $ grade                 : Factor w/ 7 levels "A","B","C","D",..: 
 $ emp_length            : Factor w/ 12 levels "< 1 year","1 year",..: 
 $ home_ownership        : Factor w/ 5 levels "MORTGAGE","NONE",..: 
 $ annual_inc            : num 
 $ verification_status   : Factor w/ 3 levels "Not Verified",..: 
 $ issue_d               : 'difftime' num 
  ..- attr(*, "units")= chr "days"
 $ purpose               : Factor w/ 15 levels "car","credit_card",..: 
 $ dti                   : num 
 $ delinq_2yrs           : num 
 $ earliest_cr_line      : 'difftime' num 
  ..- attr(*, "units")= chr "days"
 $ inq_last_6mths        : int 
 $ mths_since_last_delinq: logi 